In [4]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 44.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 데이터셋 다운로드
train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

# Null 값 제거
train_data = train_data.dropna(how = 'any')
test_data = test_data.dropna(how = 'any')

# 정규 표현식을 이용한 한글 외 문자 제거
train_data['document'] = train_data['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
test_data['document'] = test_data['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

# 불용어 처리
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
okt = Okt()
X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)
    
X_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [8]:
# 정수 인코딩
tokenizer = Tokenizer(num_words=5000) # num_words 값 설정
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# 패딩 처리
max_len = 30 # 전체 데이터의 길이를 30으로 맞춘다
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# 모델 정의
model = Sequential()
model.add(Embedding(tokenizer.num_words, 100)) # 임베딩 벡터의 차원은 100
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# 모델 학습
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, train_data['label'], epochs=10, batch_size=64)

Epoch 1/10
2344/2344 [==============================] - 193s 81ms/step - loss: 0.4033 - accuracy: 0.8130
Epoch 2/10
2344/2344 [==============================] - 194s 83ms/step - loss: 0.3503 - accuracy: 0.8443
Epoch 3/10
2344/2344 [==============================] - 197s 84ms/step - loss: 0.3284 - accuracy: 0.8549
Epoch 4/10
2344/2344 [==============================] - 200s 85ms/step - loss: 0.3132 - accuracy: 0.8636
Epoch 5/10
2344/2344 [==============================] - 202s 86ms/step - loss: 0.3019 - accuracy: 0.8695
Epoch 6/10
2344/2344 [==============================] - 203s 87ms/step - loss: 0.2920 - accuracy: 0.8754
Epoch 7/10
2344/2344 [==============================] - 203s 86ms/step - loss: 0.2829 - accuracy: 0.8802
Epoch 8/10
2344/2344 [==============================] - 205s 87ms/step - loss: 0.2744 - accuracy: 0.8841
Epoch 9/10
2344/2344 [==============================] - 204s 87ms/step - loss: 0.2658 - accuracy: 0.8886
Epoch 10/10
2344/2344 [==============================] 